In [ ]:
%pip install -r requirements.txt

In [7]:
import os
import numpy as np
import time
import constructivo as constr
import ruido as ruido
import rand_choice as aleat

In [8]:
directory_path = 'JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

In [9]:
import openpyxl

def write_xlsx(machine_order, makespan, time, name_sheet, name_file):
    # Create a new workbook and select the active sheet
    try:
        workbook = openpyxl.load_workbook(name_file)
        sheet = workbook.create_sheet(name_sheet)
    except:
        workbook = openpyxl.Workbook()
        sheet = workbook.create_sheet(name_sheet)

    # Iterate over the matrix and write its values to the sheet
    for row_index, row in enumerate(machine_order, start=1):
        for col_index, value in enumerate(row, start=1):
            sheet.cell(row=row_index, column=col_index, value=value)
    
    sheet.cell(row=row_index+1, column=1, value=makespan)
    sheet.cell(row=row_index+1, column=2, value=time)

    # Save the workbook to a file
    workbook.save(name_file)

    # Close the workbook
    workbook.close()

In [10]:
#Reading all the information
for file in directory_files:
  with open(directory_path + "/" + file, "r" ) as f:
  #with open("AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = max([sum([x[1] for x in machines[i]]) for i in machines])

  # Create job objects
  jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]
  jobs_ruido = [ruido.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1, 0.1) for i in range(n)]
  jobs_aleat = [aleat.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines_constr = [constr.Machine(i+1) for i in range(m)]
  machines_ruido = [ruido.Machine(i+1) for i in range(m)]
  machines_aleat = [aleat.Machine(i+1) for i in range(m)]

  # Record the start time
  constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()
  ruido_start_time = time.time(); ans_ruido, machine_order_ruido, _ = ruido.aleatorio(jobs_ruido, machines_ruido); ruido_end_time = time.time()
  aleat_start_time = time.time(); ans_aleat, machine_order_aleat = aleat.eleccion_aleatoria(jobs_aleat, machines_aleat); aleat_end_time = time.time()
  
  #Save the results in excel worksheets
  write_xlsx(machine_order_constr, ans_constr, constr_end_time - constr_start_time, file, "JSSP_coberndorm_Constructivo.xlsx")
  write_xlsx(machine_order_ruido, ans_ruido, ruido_end_time - ruido_start_time, file, "JSSP_coberndorm_Ruido.xlsx")
  write_xlsx(machine_order_aleat, ans_aleat, aleat_end_time - aleat_start_time, file, "JSSP_coberndorm_Aleatorio.xlsx")

  print(file, ": ", ans_constr, "    ", ans_ruido, "    ", ans_aleat, "     ", cota_inferior)

JSSP1.txt :  1462      1380      1409       977
JSSP10.txt :  2710      2568      2418       1761
JSSP11.txt :  3856      3367      3266       2760
JSSP12.txt :  3266      3146      3243       2756
JSSP13.txt :  3606      3335      3461       2868
JSSP14.txt :  3639      3417      3478       2848
JSSP15.txt :  6232      5905      6060       5464
JSSP16.txt :  5973      5724      5729       5181
JSSP2.txt :  1446      1399      1398       919
JSSP3.txt :  1865      1751      1615       1139
JSSP4.txt :  1667      1591      1637       1251
JSSP5.txt :  2175      1983      2025       1182
JSSP6.txt :  1965      1871      1903       1240
JSSP7.txt :  2335      2096      2140       1764
JSSP8.txt :  2432      2241      2195       1774
JSSP9.txt :  2499      2413      2547       1830
